In [5]:
import numpy as np
import skimage
import utils
import pathlib

In [6]:
def cumulative_sum(limit, array, element_function):
    sum = 0
    for i in range(0, limit):
        sum += element_function(i, array[i])
    return sum

# Got a zero division error
def zero_division(x, y): 
    if np.abs(y) <= 1e-8:
        return 0
    else:
        return x / y

In [7]:
def otsu_thresholding(im: np.ndarray, find_eta=False) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a boolean image
        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            find_eta: boolean, true if the eta_star should be returned also
        return:
            if find_eta
                (int, int) the computed thresholding value and eta
            else 
                (int) the computed thresholding value
            
    """
    # I will follow the seven steps from the chapter provided, and comment them along the way
    assert im.dtype == np.uint8
    L = 256 # Distinct integer intensty levels in the digital image

    # 1. Compute the normalized histogram of the input image. 
    # Denote the components of the histogram by p_i, i = 0, 1, 2, ..., L - 1
    (histogram, _) = np.histogram(im, L, (0, L - 1))
    p = histogram / np.sum(histogram) # Normalize
    assert(np.sum(p) == 1)

    # 2. Compute the cumulative sums, P_1(k), 
    # for k = 0, 1, 2, ..., L - 1,
    # using Eq. (10-49)
    P_1 = np.fromiter(map(lambda k: cumulative_sum(k + 1, p, lambda i, p_i: p_i), range(L)), np.float64)

    # 3. Compute the cumulative means, m(k), 
    # for k = 0, 1, 2, ..., L - 1, 
    # using Eq. (10-53)
    m = np.fromiter(map(lambda k: cumulative_sum(k + 1, p, lambda i, p_i: i * p_i), range(L)), np.float64)

    # 4. Compute the global mean, m_G, using Eq. (10-54)
    m_G = cumulative_sum(L - 1, p, lambda i, p_i: i * p_i)

    # 5. Compute the between-class variance term, sigma_B2(k), 
    # for k = 0, 1, 2, ..., L - 1
    # using Eq. (10-62)
    numerator = (m_G * P_1 - m) ** 2
    denominator = P_1 * (1 - P_1)
    sigma_B2 = np.fromiter(map(lambda i: zero_division(numerator[i], denominator[i]), range(L)), np.float64)

    # 6. Obtain the Otsu threshold, k_star, as the value of k for which sigma_B2(k) is maximum.
    # If the maximum is not unique, obtain k_star by averaging the values of k corresponding to the various maxima detected.
    sigma_B2_max = np.amax(sigma_B2)
    k_stars = [i for i, j in enumerate(sigma_B2) if j == sigma_B2_max]
    k_star = int(sum(k_stars) / len(k_stars))
    threshold = k_star

    # 7. Compute the global variance, sigma_G2, using Eq. (10-58), 
    # and then obtain the separability measure, eta_star, by evaluating Eq. (10-61) with k = k_star
    if find_eta:
        sigma_G2 = cumulative_sum(L, p, lambda i, p_i: (i - m_G) ** 2 * p_i)
        eta = sigma_B2 / sigma_G2
        eta_star = eta[k_star]
        return threshold, eta_star
    else:
        return threshold

In [8]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("polymercell.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: img/thumbprint.png
Found optimal threshold: 152
Saving image to: img/processed/thumbprint-segmented.png
Reading image: img/polymercell.png
Found optimal threshold: 181
Saving image to: img/processed/polymercell-segmented.png
